# Sensitivity analysis

In [ ]:
import numpy
import scipy.stats
import pandas
import matplotlib.pyplot as plt
import sympy
import SALib
plt.style.use("bmh")
from mpl_toolkits.mplot3d import Axes3D
%config InlineBackend.figure_formats=["svg"]

Funkcja Rosenbrock'a jest klasykiem w analizie niepewności i wrażliwości.

In [ ]:
def rosenbrock(x1, x2):
    return 100 * (x2 - x1**2)**2 + (1 - x1)**2

Na szczeście jest to funkcja w małej liczbie wymiarów więc narysujmy ją nad dziedziną $[-2, 2]^2$ by zobaczyć jej kształt.

In [ ]:
N = 100
fig = plt.figure()
ax = fig.gca(projection="3d")
# TODO: podziel dziedzinę na N punktów w każdym wymiarze (numpy.linspace)

X, Y = numpy.meshgrid(x, y)
ax.plot_surface(X, Y, rosenbrock(X, Y), alpha=0.8)
ax.set_facecolor("white")
plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$");

## Local sensitivity analysis

Do analizy lokalnej potrzebujemy policzyć pochodne lokalne funkcji Rosenbrock'a.

In [ ]:
from sympy.interactive import printing
printing.init_printing(use_latex="mathjax")

In [ ]:
x1 = sympy.Symbol("x1")
x2 = sympy.Symbol("x2")
rosen = 100 * (x2 - x1**2)**2 + (1 - x1)**2
# TODO: policzmy pochodne powyższej funkcji po x1 i x2

In [ ]:
d1

In [ ]:
d2

Funkcja wygląda na płaską w pobliżu $(0, 0)$; Możemy to sprawdzić za pomocą podstawienia.

In [ ]:
d1.subs({x1: 0, x2: 0})

In [ ]:
d2.subs({x1: 0, x2: 0})

Jak widać powyżej, funkcja w tamtym miejscu nie jest zbytnio rosnąca czy malejąca więc jest tam mało wrażliwa na zmiany wejścia.

Funkcja jest bardziej stroma (czyli będzie tam większa wrażliwość lokalna) w pobliżu punktu $(-2, -2)$.

In [ ]:
d1.subs({x1: -2, x2: -2})

In [ ]:
d2.subs({x1: -2, x2: -2})

W $(-2, 2)$ podobnie.

In [ ]:
d1.subs({x1: -2, x2: 2})

In [ ]:
d2.subs({x1: -2, x2: 2})

Przy użyciu SciPy'a możemy obliczyć minimum dla całej funkcji nad dziedziną $[-2, 2]^2$
. Użyj funkcji `scipy.optimize.fmin`z punktem startowym `[2, 2]`.

In [ ]:
import scipy
# TODO

## Global sensitivity analysis

Do globalnej analizy wrażliwości użyjemy framework'u SALib (https://github.com/jdherman/SALib)

In [ ]:
from SALib.sample import saltelli
from SALib.analyze import sobol

In [ ]:
N = 1000
problem = {
    "num_vars": 2, 
    "names": ["x1", "x2"], 
    "bounds": [[-2, 2], [-2, 2]]
}
sample = saltelli.sample(problem, N)
Y = numpy.empty([sample.shape[0]])
# wypełnij tablicę Y wartościami zwracanymi przez funkcję Rosenbrock'a dla wygenerowanych próbek

sensitivity = sobol.analyze(problem, Y)
sensitivity["S1"]

`S1` zawiera wskaźnik wrażliwości pierwszego rzędu, który mówi jak bardzo $x_1$ i $x_2$ wpływają na ogólną zmienność wyjściową funkcji nad dziedziną $[-2, 2]^2$. 

**Interpretacja**: możemy zauważyć, że $x_1$ (dla którego indeks wrażliwości wynosi ok. 0.5) wpływa na około połowę całkowitej niepewności wyniku oraz ma prawie dwa razy większy wpływ niż $x_2$. 

In [ ]:
sensitivity["ST"]

`ST` zawiera wskaźniki łączne, które obejmują "efekty interakcji" z innymi zmiennymi. 

**Interpretacja**: Całkowita wrażliwość zmiennej $x_1$ (ok. 0.7) wskazuje, że znaczna część (ok. 20%) naszej całkowitej niepewności wyjściowej wynika z interakcji $x_1$ z innymi zmiennymi wejściowymi. Ponieważ istnieją tylko dwie zmienne wejściowe, wiemy, że ten efekt interakcji musi być związany z $x_2$.

Niektóre metody analizy wrażliwości są również w stanie dostarczyć wskaźniki wrażliwości drugiego i trzeciego rzędu. Indeks drugiego rzędu $s_{i, j}$ informuje o poziomie efektów interakcji między $x_i$ a $x_j$ (efekty interakcji są większe niż zero, gdy twoja funkcja jest nieliniowa: czułość parametru $i$ może następnie zależeć od wartości parametru $j$). Indeks trzeciego rzędu $s_{i, j, k}$ informuje o poziomie interakcji między trzema parametrami $x_i$, $x_j$ i $x_k$.

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5006244/

## Zadanie

Wykonajmy analizę dla funkcji Ishigami, która jest przykładem bardzo nieliniowej funkcji i w dobry sposób testuje metody przeprowadzania analizy wrażliwości.

In [ ]:
def ishigami(x1, x2, x3) -> float:
    return numpy.sin(x1) + 7*numpy.sin(x2)**2 + 0.1 * x3**4 * numpy.sin(x1)

Analizę wykonajmy nad dziedziną $[-\pi, \pi]^3$.

In [ ]:
# TODO: zdefiniuj problem jak przy funkcji Rosenbrock'a, stwórz próbki z wykorzystaniem fast_sampler'a oraz oblicz dla nich wartości funkcji ishigami


In [ ]:
x = sample[:, 0]
y = sample[:, 1]
z = sample[:, 2]
c = Y

In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.tri as mtri

do_random_pt_example = True;

index_x = 0; index_y = 1; index_z = 2; index_c = 3;
list_name_variables = ['x', 'y', 'z', 'c'];
name_color_map = 'seismic';
triangles = mtri.Triangulation(x, y).triangles;

choice_calcuation_colors = 1;
if choice_calcuation_colors == 1: # Mean of the "c" values of the 3 pt of the triangle
    colors = np.mean( [c[triangles[:,0]], c[triangles[:,1]], c[triangles[:,2]]], axis = 0);
elif choice_calcuation_colors == 2: # Mediane of the "c" values of the 3 pt of the triangle
    colors = np.median( [c[triangles[:,0]], c[triangles[:,1]], c[triangles[:,2]]], axis = 0);
elif choice_calcuation_colors == 3: # Max of the "c" values of the 3 pt of the triangle
    colors = np.max( [c[triangles[:,0]], c[triangles[:,1]], c[triangles[:,2]]], axis = 0);
fig = plt.figure();
ax = fig.gca(projection='3d');
triang = mtri.Triangulation(x, y, triangles);
surf = ax.plot_trisurf(triang, z, cmap = name_color_map, shade=False, linewidth=0.2);
surf.set_array(colors); surf.autoscale();

#Add a color bar with a title to explain which variable is represented by the color.
cbar = fig.colorbar(surf, shrink=0.5, aspect=5);
cbar.ax.get_yaxis().labelpad = 15; cbar.ax.set_ylabel(list_name_variables[index_c], rotation = 270);

# Add titles to the axes and a title in the figure.
ax.set_xlabel(list_name_variables[index_x]); ax.set_ylabel(list_name_variables[index_y]);
ax.set_zlabel(list_name_variables[index_z]);
plt.title('%s in function of %s, %s and %s' % (list_name_variables[index_c], list_name_variables[index_x], list_name_variables[index_y], list_name_variables[index_z]) );

plt.show()

In [ ]:
# Analiza fast wartości funkcji ishigami obliczonych dla wygenerowanych próbek
from SALib.analyze import fast
sensitivity = fast.analyze(problem, Y, print_to_console=True)

## Inne przykłady użycia metod z biblioteki SALib

In [ ]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

from SALib.analyze import morris
from SALib.sample.morris import sample
from SALib.test_functions import Ishigami, Sobol_G
from SALib.util import read_param_file
from SALib.plotting.morris import horizontal_bar_plot, covariance_plot, sample_histograms

# Read the parameter range file and generate samples
problem = read_param_file('Sobol_G.txt')

# Generate samples
param_values = sample(problem, N=1000, num_levels=4)

# Run the "model" -- this will happen offline for external models
Y = Sobol_G.evaluate(param_values)

# Perform the sensitivity analysis using the model output
# Specify which column of the output file to analyze (zero-indexed)
Si = morris.analyze(problem, param_values, Y, conf_level=0.95, 
                    print_to_console=True,
                    num_levels=4, num_resamples=100)

print ('Convergence index:', max(Si['mu_star_conf']/Si['mu_star']))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
horizontal_bar_plot(ax1, Si,{}, sortby='mu_star', unit=r"tCO$_2$/year")
covariance_plot(ax2, Si, {}, unit=r"tCO$_2$/year")
plt.show()

In [ ]:
fig2 = plt.figure()
sample_histograms(fig2, param_values, problem, {'color':'y'})
plt.show()

## Do domu

Przeprowadzić analizę wrażliwości dla wymyślonego przez siebie modelu (jakaś funkcja przyjmująca 2 lub 3 argumenty).